# Writing high performance functions on Python using numba #
------
<div style="text-align: right"> <font size="4">- Sundar</font>  </div>


## Introduction

Are you writing codes on native python (not using libraries such as tensorflow/pytorch)? Are you writing non-ML codes? Are you interested to write your own functions that can exploit GPU capabilities?

If your answer to any of the above questions is yes, this notebook will serve to help you.


**Objectives:** Get started with learning to write functions

- that run as fast as C/Fortran functions
- that can be computed parallely on GPUs

This notebook contains a brief introduction to writing such python codes using numba to help you get kickstarted with high performance python computing.

<br>
<div class="alert alert-danger">
<b>Disclaimer:</b>  
   This tutorial is not meant to teach you to write codes of lower complexity (meaning that this will not make you a "smarter" programmer), instead this gives an overview of some recent developments in Python that use CPU/GPU resources more efficiently. I am  new to numba and programming on GPUs myself and  am by no means an expert in any programming language whatsoever. I could be wrong about some statements I make in this notebook, so please do your own research as well!
</div>



### What do I need?

- A PC/laptop (duh!)
- Python installation (3+ preferably) and basic knowledge of python.  If you are using either finback or humpback servers at UCLA, python 3+ comes pre-installed and can be accessed using the shell command "python3".
- Jupyter notebook installation. If you are using either finback or humpback it takes some effort to set this up -- please check out [Yahya's tutorial](https://www.google.com/url?q=https://www.evernote.com/shard/s168/sh/af5824e5-3f41-400b-b872-04b8e5cf8d07/4e57c09a3811e7f1f79ac2c0590d4a29&sa=D&source=hangouts&ust=1579638605374000&usg=AFQjCNGvDsZu1oMXky6XMB1IwsimJ5ZNWA) for installing and accessing jupyter notebooks on finback/humpack.
- [Numba installation](https://nyu-cds.github.io/python-numba/setup/). If you are using either finback or humpback, numba comes preinstalled so you need to do nothing.
- CUDA drivers for GPU programming (you do not need this if you are not programming on GPUs). I do not know much about this but please check [NVIDIA's page](https://docs.nvidia.com/cuda/cuda-quick-start-guide/index.html). If you are using one of our servers, you might have to install the correct version of cudatoolkit since the default version cuda 10.1 was not supported with numba and I had to do "conda install cudatoolkit=10.0" to get it working.
- Most importantly, enthusiasm to learn to write better programs and some patience, since it requires lots of typing.

<br>
<div class="alert alert-warning">
<b>Note:</b>  the easiest way to make the set-up is to install the latest distribution of Anaconda and  <a href="https://nyu-cds.github.io/python-numba/setup/">run these commands</a>.
</div>

You can contact me in case you have trouble getting it set up and running.

### Are there more reliable resources on numba and CUDA python programming?

Here are some resources which helped me get started with numba and most of the content here is regurgitated from these sources:
 
- Very user friendly [first article](https://nyu-cds.github.io/python-numba/) on numba  .
- [Official numba documentation](http://numba.pydata.org/).
- [Excellent article](https://thedatafrog.com/en/cuda-kernel-python/) on writing CUDA kernels with numba. Also has a simple example of writing 2D Kernels (we will write a simple 1D kernel in this notebook).
- [Video](https://www.youtube.com/watch?v=1AwG0T4gaO0) giving a comprehensive overview of CPU computing with numba.


## Contents:

This notebook will try to teach through examples, so it is highly suggested that you go through it cell-by-cell and not miss anything.

1. FAQs.
2. Accelerating Python functions with numba on CPUs.
3. Parallelizing python codes to run on GPUs with numba and CUDA.

## FAQs

**What is numba?**

Crude answer: Python is not a "compiled" language, rather it is an "interpreted" language -- what this means is that unlike C, python codes are not compiled (converted) into native machine codes, but instead they are converted into something called "bytecodes" which are interpreted at runtime. As you might have guessed, this is a reason Python is much slower than C/C++. There is much more to compiled vs interpreted code which I am omitting since I don't know much about it either (LOL); in fact, a language itself can't be categorized into compiled or interpreted, it is the specific implementation of that language which decides whether it generates a native code or a bytecode.

Numba is a compiler for python that translates a python code into a native machine code. It falls under the umbrella of just-in-time (JIT) compilers for python. However, writing good compilers for python is supposedly very hard (it is a trade-off for the user friendliness of writing in python).

*TLDR: Numba compiles python codes into machine codes to make it run faster.*

**Can I use numba with any python code?**

NO, with capital N-O. Numba is being actively developed so maybe in a few years it might be possible. But numba works with only a small fraction of python datatypes. It is HIGHLY recommended that you use only numpy arrays when using numba (avoid lists, dictionaries etc.). However, this suffices for a majority of the scientific computing tasks.

*TLDR: Use only numpy arrays as data types when writing functions with numba.*

**Can I use numpy functions in conjunction with numba?**

Again, numba is still in active development, so not all features are available. In my limited experience, it looks like numpy functions (especially ufuncs) are supported with CPU computation, however they are not supported with GPU computation. The safest bet is to use math module in python and compile with numba.

*TLDR: Use the math module in python for invoking functions (like sin, cos, exp etc.).*

**Will this tutorial help me get started with numba and CUDA programming straight-away?**

Probably not. This is meant to serve as a beginner level introduction to numba and CUDA programming and contains very simple examples. The goal of this notebook is to let you know of the capabilities of numba so you can read more and equip yourself with the necessary tools needed to use numba productively.

**What are my PC specs?**

All the outputs here have been generated with the humpback server at UCLA. This has a 10 core Intel i9 processor at 3.1 GHz supported by 2 Titan RTX GPUs. However, when we run the codes on CUDA, only one GPU is active at a time (I am not sure how to exploit both the GPUs with numba). 

## Part 1. Accerating functions on CPU with numba

First let us check if numba is properly installed. Executing the following cell should show the version of numba installed in the system.

In [1]:
import numba
numba.__version__

'0.47.0'

All good. Importing essential modules now.

In [2]:
import numpy as np
import math

### The computing task

Since this notebook serves as an introductory tutorial, we will stick to a very simple task throughout. When writing actual (possibly complicated) codes, it is possible that the speedups we obtain with numba and CUDA might be orders of magnitude more. 

We consider this simple  task: computing the pdf of a zero mean, unit variance gaussian for a given set of points.

$$f(x) = \frac{1}{\sqrt{2\pi}}\exp\left(\frac{-x^2}{2} \right).$$

So our computing task would be to calculate this function for each point in the array "x" which contains a million entries.

In [3]:
###### Generating the array x #######

n = 1000000  #size of array x
x = np.random.normal(0,1,n)    # generating a million random numbers

###  Naive Python implementation with loops.

Let us first do a naive python implementation. Those of you experienced with Python are probably shaking your head vigorously right now, screaming at me to use numpy. However, we do this first anyway for a baseline comparison.

In [4]:
#### Naive python implementation with loops ####

def gaus_pdf(x): 
    
    y = np.empty(x.shape)
    
    for i in range(x.shape[0]):             # we loop over the entries of x
        y[i] = 1/math.sqrt(2*math.pi) * math.exp(-x[i]**2/2)
        
    return y

In [5]:
true_answer = gaus_pdf(x)  # we store the answer for checking the correctness
                           # of our other implementations

Let us compute the time it takes using the "timeit" ipython magic function 

In [6]:
%timeit gaus_pdf(x)

865 ms ± 8.59 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Well, it took ~800 ms on an average for this task, naively. Let us see how far we can push this to.

### Numpy implementation

Rule #1 of programming with Python is to avoid loops as and when necessary (or that's what we have been taught), and use tricks like broadcasting along with scientific computing packages like numpy and scipy which are largely written in C and work excellently for vectorized operations. So we implement a numpy'ic version of the computing task, which is supposed to be very efficient.

We remark that our computing task is very simple, so a straightforward numpy implementation is possible. However, many of the computing tasks we encounter in practice are way more complicated so exploiting numpy may not always be feasible or easy to do.

In [7]:
def gaus_pdf_numpy(x):
    return 1/math.sqrt(2*math.pi) * np.exp(-x**2/2) # notice that invoking np.exp(x) makes
                                                    # the function vectorized

In [8]:
np.allclose(gaus_pdf_numpy(x),true_answer) # check to verify correctness of the function

True

Computing the time it takes with numpy

In [9]:
%timeit gaus_pdf_numpy(x)

14.3 ms ± 38.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


As expected, using numpy is roughly 60X faster than the naive implementation with loops.

### Simple numba implementation

We now use numba. The important component to use is the decorator called $\texttt{jit}$ for just-in-time compilation. This decorator compiles the function to which it is applied to generate fast machine code, instead of intermediate bytecode.

**Decorator**: decorators are functions which take a function as an input and produce another function (actually the same function with a modified behavior) as output. In our case, the @jit decorator produces a more "efficient" implementation of our function as the output. Using the decorator @jit on a function "f" is equivalent to invoking jit()(f). However, for many cases, it is convenient to use jit as a decorator instead.



We now use the decorator with the naive python implementation. We are able to do this for the following two reasons:
- The data types (inputs, outputs and intermediate variables) are all numpy arrays.
- The only math operations used are imported from the math module. Numba supports a majority of the math functions in python math modules.

We now create a "jitted" function from the naive python implementation. We do this by adding just a single line (adding the decorator) to the earlier function.

In [10]:
from numba import jit              # importing jit

@jit(nopython = True)              # the jit decorator tells numba to compile the function below

def gaus_pdf_numba(x):             # same naive python implementation as before
    y = np.empty(x.shape)
    for i in range(x.shape[0]):
        y[i] = 1/math.sqrt(2*math.pi) * math.exp(-x[i]**2/2)
    return y

Notice that we gave a parameter to jit called "nopython" and set it to True -- this forces numba to compile the code into machine code. Forcing no python mode to be true will result in an error if for some reason numba cannot compile the python code into machine code. If nopython is set to false, then numba will not throw an error in such cases, however you won't see speedups since the machine code is not generated.

**Note:** Always set nopython = True, at least when getting started with numba. Once you get acquainted with numba, you will encounter situations where you want to call a function from a jitted function. And as of now, you can only call jitted functions from another jitted function, so in these situations you might want to jit a function even if you don't acheive speedups.

In [11]:
np.allclose(gaus_pdf_numba(x),true_answer)

True

OK! The jitted function gives the correct answer. Now to test the time.

In [12]:
%timeit gaus_pdf_numba(x)

11.5 ms ± 16.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


NOT BAD! This is a tad faster than even the numpy implementation, which uses C and is highly optimized. However, this speedup does not justify the use of jit whatsoever. So now, we show how to use parallelization (with CPU multicores) with numba to acheive further speedup.

Also keep in mind that our task was very simple so a straightforward numpy implementation was possible. However, many of the tasks we deal with cannot be easily written using numpy functions and in these situations numba is extremely useful.

### Easy CPU parallelization with numba

In [13]:
from numba import prange               # prange is a function which tells numba
                                       # which loops needs to be parallelized

@jit(nopython = True, parallel = True) # setting parallel = True for CPU parallelization

def gaus_pdf_numba_parallel(x): 
    
    y = np.empty(x.shape)
    
    for i in prange(x.shape[0]):       # this loop can be parallelized since each y[i] can be
                                       # independently computed
        y[i] = 1/math.sqrt(2*math.pi) * math.exp(-x[i]**2/2)
    return y

In [14]:
# checking the correctness
np.allclose(gaus_pdf_numba_parallel(x),true_answer)

True

In [15]:
%timeit gaus_pdf_numba_parallel(x)

1.17 ms ± 12.3 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


This is roughly a 10X speedup w.r.t numpy and 800X speedup w.r.t a naive python implementation.

<div class="alert alert-info">
<b>Summary of part 1:</b>  
<ul>
  <li>numba provides jit, a just-in-time compiler for python.</li>
  <li>Always use numpy arrays as datatype for numba, and use functions from math modules for math operations.</li>
  <li>Numba provides easy CPU parallelization with prange.</li>
</ul>
</div>

## Part 2: Python CUDA programming with numba

We stick to the same task for GPU programming. Rewriting this function to make it GPU compatible requires a little bit of extra typing. Moreoever, GPU programming is also specific to the GPU at hand and requires a little bit of knowledge on the functioning of the GPUs in general -- here we will learn the bare minimum needed in order to get us rolling. I refer to [NVIDIA CUDA programming guide for C++](https://docs.nvidia.com/cuda/cuda-c-programming-guide/) for a comprehensive overview of this subject.

### Some background on GPUs

We first start with some nomenclature:

| Term | Meaning |
| --- | :---: |
| **Host** | The CPU | 
| **Device** | The GPU | 
|**Kernel** | A GPU function launched by the host and executed on the device | 
| **Device function** | a GPU function executed on the device which can only be called |
| | from the device (i.e. from a kernel or another device function) | 

The most important image to keep in mind is this:

![Image of CUDA_programming model](https://docs.nvidia.com/cuda/cuda-c-programming-guide/graphics/grid-of-thread-blocks.png)

In order to parallelize our code and run it on hundreds or thousands of CUDA cores, we first launch a "grid". This grid contains many "blocks" and each block contains many "threads". Our parallelization is accomplished by these threads. In the picture above, the threads can be visualized as a 2-D array (a matrix) with 6 rows and 12 columns. The idea is as follows: say we want to parallelize a computation which outputs a matrix, and each entry of this matrix can be computed separately. We program this grid of block threads to compute a $6\times 12$ submatrix of entries at a time (each thread computes one entry of the matrix independently) and slide it across the entire matrix to compute all the entries. Typically we launch a bigger grid of threads (we will briefly discuss good choices for the grid size later).


### Writing a CUDA kernel for the example computation task

Let's get into parallelizing our code to run it on the GPUs. First we import the cuda programming module.


In [16]:
from numba import cuda

Now, we do not want to cram all the logic into the CUDA kernel. Hence, we write a device function that will do parts of the computation. Keep in mind that device functions can be called only by other device functions or kernels.

In [17]:
@cuda.jit(device = True)            # notice that we compile with the decorator cuda.jit
                                    # also device = True defines this as a device function

def compute_pdf(x):                 # we define the scalar version of the function, the
                                    # vectorization is handled by the CUDA kernel
    return 1/math.sqrt(2*math.pi) * math.exp(-x**2/2)

Nothing fancy at this point, just write a function to be used by the device as you would write typical python functions. Note that we used functions from the python math module (using numpy functions will throw and error).

Let us now first write the CUDA kernel and then we will explain what we did. 

In [18]:
@cuda.jit  # the decorator to compile a cuda kernel

def gaus_pdf_cuda(x,y):
    
    start = cuda.grid(1)                        # gets the start of the grid
    stride = cuda.gridsize(1)                   # getting the size of the grid
    
    for idx in range(start,y.shape[0],stride):  # this is the sliding (or striding) of the matrix to cover the output
        y[idx] = compute_pdf(x[idx])

There are three key aspects to the above kernel:

- CUDA kernels can't (yet) return a value. Instead they do in-place modifications. That is why we need to give the output array y as an argument and then modify it in the kernel.
- We assume that we launch a 1-D array of threads (think of the earlier figure, but the blocks and threads are arranged in a 1-D array like fashion)
- We stride (or slide) over the input/output array to cover the entire length. Otherwise, only a few entries (equal to the size of the grid) of the output would be computed.

This is accomplished by the following:

- The y[idx] = compute_pdf(x[idx]): this will compute the pdf for one grid of entries.
- the for loop which does the striding to cover all the entries.

Now let us see how to call this kernel function from the host. Recall that our input is already stored in "x".

In [19]:
d_x = cuda.to_device(x)              # move the data from the host to the device
d_y = cuda.device_array_like(x)      # create an array in the device for storing the output

blocks_per_grid = 256                # define a 1-D block of grids
threads_per_block = 512              # number of threads in a block
 
gaus_pdf_cuda[blocks_per_grid,threads_per_block](d_x,d_y)   # call kernel with the defined grid
cuda.synchronize()                                          # tells the CPU to wait till GPU finishes

y = d_y.copy_to_host()               # copy the answer back to the host

A lot of things happened here:
- We first moved the input data into the device
- We then created an array for the kernel to store the output
- Defined a grid of 256 blocks (arranged in a 1-D fashion) and 512 threads per block
- Call the kernel
- Ensure that the CPU is synchronized (i.e. does not run in the background and do other stuff)
- Retrieve the data

A natural question that should arise here is why the particular choices of block and thread sizes. The answer to this question is not simple and it takes further understanding of the CUDA programming model  (which I don't have a good grasp of yet) to fully comprehend these numbers. However, here is a good rule to keep in mind -- the number of blocks per grid should be 2X-4X the number of streaming multiprocessors (SMs) in the GPU. I am running the codes on Titan RTX with 72 SMs. The number of threads in a block should be a multiple of 32 -- it might help to vary it from 128 to 512 and test to see which choice gives the best performance for your particular need.

In [20]:
print(np.allclose(true_answer,y))    # answer verification

True


The code works, now let us time it. Remember to include the synchronization in our time test module as well.

In [21]:
%timeit gaus_pdf_cuda[blocks_per_grid,threads_per_block](d_x,d_y); cuda.synchronize()

230 µs ± 3.24 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


This is a 5X speedup even compared to the CPU parallelized version. In fact, larger input sizes would give even more speedup.

In [22]:
n = 10000000 
x = np.random.normal(0,1,n)    # generating 10 million random numbers

%timeit gaus_pdf_numba_parallel(x)

d_x = cuda.to_device(x)              # move the data from the host to the device
d_y = cuda.device_array_like(x)      # create an array in the device for storing the output

%timeit gaus_pdf_cuda[blocks_per_grid,threads_per_block](d_x,d_y); cuda.synchronize()

18.2 ms ± 288 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
959 µs ± 1.73 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


You can see that larger input sizes give more speedup with a GPU as compared to even the best implementation on the CPU!! However, someone of you may have noticed that I did not include the time it takes for transferring the data from the host to the device and back. Let us include that as well and recompute the time.

In [23]:
# defining a wrapper function for use with timeit function
def gaus_pdf_cuda_complete(x):
    d_x = cuda.to_device(x)              # move the data from the host to the device
    d_y = cuda.device_array_like(x)      # create an array in the device for storing the output

    blocks_per_grid = 256                # define a 1-D block of grids
    threads_per_block = 512              # number of threads in a block

    gaus_pdf_cuda[blocks_per_grid,threads_per_block](d_x,d_y)   # call kernel with the defined grid
    cuda.synchronize()                                          # tells the CPU to wait till GPU finishes

    y = d_y.copy_to_host()               # copy the answer back to the host
    return y

In [24]:
%timeit gaus_pdf_cuda_complete(x)

33.8 ms ± 605 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


This is more than the time it takes with CPU parallelization!! The reason GPUs don't benefit much in this case is because most of the GPU time is spent on transferring huge data back and forth, and the computation task itself comprises only a small fraction of the total time.

GPUs really shine when the "arithmetic intensity" is high, i.e., when a lot of math operations are involved -- in this case, data transfer would consume much less time that the actual computing task.  The computing task in our simple example was not nearly complicated enough to warrant the use of GPUs since the data transfer served as the bottleneck.

<div class="alert alert-info">
<b>Summary of part 2:</b>  
<ul>
  <li>numba provides cuda.jit, a decorator for CUDA kernels and device functions for use with NVIDIA GPUs.</li>
  <li>CUDA kernels can't explicitly return values, instead the output has to be passed as an argument which should then be modified in the kernel.</li>
  <li>CUDA device functions can be written to support the kernels.</li>
     <li>CUDA programming is useful only when the operations are complicated enough to justify data transfer back and forth between the host and the device.</li>
</ul>
</div>
